In [4]:
from openpyxl import load_workbook
import re
import pymysql

In [5]:
wb = load_workbook(filename = 'duplicate_removal_final.xlsx')

In [6]:
 ws = wb.get_sheet_by_name(name = 'duplicate_removal_final')

In [7]:
GREEN = 'FFC6EFCE'

In [8]:
first = True
row_itr = ws.iter_rows()
col_names = [cell.value for cell in next(row_itr)]

replace_dict = {}

for row in row_itr:
    replace_oids = []
    base_oid = None

    for col_name, cell in zip(col_names, row):
        
        if col_name == 'oid':
            base_oid = cell.value
        elif re.match(r'oid_(\d){1,4}', col_name):
            if cell.value is None:
                break
            elif cell.fill.start_color.index == GREEN:
                replace_oids.append(cell.value)
                
    replace_dict[base_oid] = replace_oids

In [ ]:
def update_oid(cursor, old_oid=None, new_oid=None):
    tables = [
    ('Gift', 'oid'), 
    ('Contribution', 'oid'),
    ('LobbyistDirectEmployment', 'lobbyist_employer'),
    ('LobbyingContracts', 'lobbyist_employer'),
    ('LobbyistRepresentation', 'oid'),
    ('GeneralPublic', 'oid'),
    ('Behests', 'payee'),
    ('OrgAlignments', 'oid'),
    ('LobbyistEmployer', 'oid')
    ]
    
    for table, field in tables:
        query = '''UPDATE {table}
                   SET {field} = {new_oid}
                   WHERE {field} = {old_oid}'''
        d = {'table': table,
             'field': field,
             'new_oid': new_oid,
             'old_oid': old_oid}
        
        try:
            cursor.execute(query.format(**d))
        except pymysql.IntegrityError as e:
            err_num = e.args[0]
            if err_num == 1062:
                query = 'DELETE FROM {table} WHERE {field} = {old_oid}'
                cursor.execute(query.format(**d))
            else:
                print(e)
                print('table', table)
                print('field', field)
                print('old_oid', old_oid)
                print('new_oid', new_oid)
                assert False
    
    query = 'DELETE FROM OrgConceptAffiliation WHERE old_oid = %s'
    cursor.execute(query % old_oid)
    
    query = 'DELETE FROM Organizations WHERE oid = %s'
    cursor.execute(query % old_oid)

In [18]:
CONN_INFO = {'host': 'dddb2016-mysql5-7-11.chzg5zpujwmo.us-west-2.rds.amazonaws.com',
             'port': 3306,
#              'db': 'EricTest',
             'db': 'DDDB2016Aug',
             'user': 'awsDB',
             'passwd': 'digitaldemocracy789'}

In [ ]:
with pymysql.connect(**CONN_INFO) as cursor:

    for new_oid, values in replace_dict.items():
        for old_oid in values:
            update_oid(cursor, old_oid=old_oid, new_oid=new_oid)


In [16]:
values = ''
for k, v in replace_dict.items():
    for oid in v:
        values += ' (' + str(oid) + '),'

values = values[:-1]    

In [21]:
with pymysql.connect(**CONN_INFO) as cursor:
    stmt = """insert into DeprecatedOrganization
                (oid) 
               values
               %s"""

    cursor.execute(stmt % values)